対象設定

In [1]:
name="mimikkyu"

デフォルトマーカー位置設定

In [2]:
import numpy as np 
import cv2
import glob
import os

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
dst=cv2.imread( "./img/"+"/md_"+str(name)+".png") 
def_corners, def_ids, rejectedImgPoints = aruco.detectMarkers(dst, dictionary)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]

def_pts=np.zeros((8,4,2),dtype = np.float32)

for i in count2:
    for j in count2:
        if j ==def_ids[i][0]:
            for k in count:
                def_pts[j][k][0]=def_corners[i][0][k][0]
                def_pts[j][k][1]=def_corners[i][0][k][1]
 
        
height, width, color = dst.shape


def_bord_pts=np.zeros((16,2),dtype = np.float32)
def_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
def_pts
for i in count:
    for j in count:
        def_bord_pts[4*i+j][0]=def_pts[i][j][0]
        def_bord_pts[4*i+j][1]=def_pts[i][j][1]    

for i in count:
    for j in count:
        def_pro_pts[4*i+j][0]=def_pts[4+i][j][0]
        def_pro_pts[4*i+j][1]=def_pts[4+i][j][1]


#print("def_pts")
#print(def_pts)
#print("def_pro_pts")
#print(def_pro_pts)
#print("def_bord_pts")
#print(def_bord_pts)

貼り付け関数

In [3]:
def cvpaste(img, imgback, x, y, angle, scale):  
    # x and y are the distance from the center of the background image 

    r = img.shape[0]
    c = img.shape[1]
    rb = imgback.shape[0]
    cb = imgback.shape[1]
    hrb=round(rb/2)
    hcb=round(cb/2)
    hr=round(r/2)
    hc=round(c/2)

    # Copy the forward image and move to the center of the background image
    imgrot = np.zeros((rb,cb,3),np.uint8)
    imgrot[hrb-hr:hrb+hr,hcb-hc:hcb+hc,:] = img[:hr*2,:hc*2,:]

    # Rotation and scaling
    M = cv2.getRotationMatrix2D((hcb,hrb),angle,scale)
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))
    # Translation
    M = np.float32([[1,0,x],[0,1,y]])
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))

    # Making mask
    imggray = cv2.cvtColor(imgrot,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(imggray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of the forward image in the background image
    img1_bg = cv2.bitwise_and(imgback,imgback,mask = mask_inv)

    # Take only region of the forward image.
    img2_fg = cv2.bitwise_and(imgrot,imgrot,mask = mask)

    # Paste the forward image on the background image
    imgpaste = cv2.add(img1_bg,img2_fg)

    return imgpaste


メイン関数

In [4]:
import cv2
import numpy as np
import time

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
img=cv2.imread("img/"+str(name)+".jpg")
imgback = cv2.imread("img/"+str(name)+".jpg")

height,width,color=imgback.shape

angle=0
scale=0.2

ar4=cv2.imread("ARUCO/blue_ar04.png")
ar5=cv2.imread("ARUCO/blue_ar05.png")
ar6=cv2.imread("ARUCO/blue_ar06.png")
ar7=cv2.imread("ARUCO/blue_ar07.png")

cap = cv2.VideoCapture(1)

rt_pts=np.zeros((8,4,2),dtype = np.float32)
rt_bord_pts=np.zeros((16,2),dtype = np.float32)
rt_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]
count3=[4,5,6,7]

mat1, mask1 = cv2.findHomography(def_pro_pts, def_pro_pts, cv2.RANSAC,1)#初期設定（変更なし）
mat2, mask2 = cv2.findHomography(def_bord_pts, def_bord_pts, cv2.RANSAC,1)
frag=1#待機時間管理

while(True):

    t0 = time.time() #時間計測開始   
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    frame2=cv2.warpPerspective(img, mat2, (3*width, 3*height))
    frame3=cv2.warpPerspective(frame2, mat1, (width, height))#カメラから見たプロジェクタ上の座標→カメラから見たボード上の座標
    

    imgpaste = cvpaste(ar4, frame3, 0, -90, angle, scale)#AR貼り付け
    imgpaste2 = cvpaste(ar5, imgpaste, 180, 0, angle, scale)
    imgpaste3 = cvpaste(ar6, imgpaste2, 0, 90, angle, scale)
    imgpaste4 = cvpaste(ar7, imgpaste3, -180, 0, angle, scale)

    
    cv2.namedWindow('screen', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('screen',imgpaste4)
  

    new_corners, new_ids, rejectedImgPoints = aruco.detectMarkers(frame, dictionary)
    
    if not new_corners:
        pass
        
    else:
        rt_bord_pts=np.zeros((16,2),dtype = np.float32)#reset
        rt_pro_pts=np.zeros((16,2),dtype = np.float32)
        def_bord_pts2=np.zeros((16,2),dtype = np.float32)
        def_pro_pts2=np.zeros((16,2),dtype = np.float32)
        num=new_ids.size-1
        
        for i in count2:
            for j in count2:
                if j ==new_ids[i][0]:
                    for h in count:
                        rt_pts[j][h][0]=new_corners[i][0][h][0]                           
                        rt_pts[j][h][1]=new_corners[i][0][h][1]
            if num==i:
                   break
            

        k=0
        for i in count:
            if rt_pts[i][0][0]==rt_pts[i][0][1]==0:
                for j in count:
                    def_bord_pts2=np.delete(def_bord_pts2,-1,0)
                    rt_bord_pts=np.delete(rt_bord_pts,-1,0)
                    
            else:
                for j in count:
                    def_bord_pts2[4*k+j][0]=def_bord_pts[4*i+j][0]
                    def_bord_pts2[4*k+j][1]=def_bord_pts[4*i+j][1] 
                    rt_bord_pts[4*k+j][0]=rt_pts[i][j][0]
                    rt_bord_pts[4*k+j][1]=rt_pts[i][j][1] 

                k=k+1
                        
        k=0
        for i in count:
            if rt_pts[i+4][0][0]==rt_pts[i+4][0][1]==0:
                for j in count:
                    def_pro_pts2=np.delete(def_pro_pts2,-1,0)
                    rt_pro_pts=np.delete(rt_pro_pts,-1,0)
                    
            else:
                for j in count:
                    def_pro_pts2[4*k+j][0]=def_pro_pts[4*i+j][0]
                    def_pro_pts2[4*k+j][1]=def_pro_pts[4*i+j][1]
                    rt_pro_pts[4*k+j][0]=rt_pts[i+4][j][0]
                    rt_pro_pts[4*k+j][1]=rt_pts[i+4][j][1]
                k=k+1
            if  rt_pro_pts.size==0 or rt_bord_pts.size==0 : 
                pass
            
            elif frag>=30:#待機時間管理
                mat1, mask1 = cv2.findHomography(rt_pro_pts, def_pro_pts2, cv2.RANSAC,1)
                mat2, mask2 = cv2.findHomography(def_bord_pts2, rt_bord_pts, cv2.RANSAC,1)
    #print(cap.get(cv2.CAP_PROP_FPS))


    if cv2.waitKey(1) &0xFF == ord('q'):
        break
    frag=frag+1
    t1 = time.time()                # 計測終了時間
    elapsed_time = float(t1 - t0)   # 経過時間
   # print(elapsed_time)             # 経過時間を表示
    
cv2.destroyAllWindows()

In [74]:
def_bord_pts2

array([[  5.,   7.],
       [ 34.,   7.],
       [ 34.,  36.],
       [  5.,  36.],
       [365.,   7.],
       [394.,   7.],
       [394.,  36.],
       [365.,  36.],
       [  5., 187.],
       [ 34., 187.],
       [ 34., 216.],
       [  5., 216.],
       [365., 187.],
       [394., 187.],
       [394., 216.],
       [365., 216.]], dtype=float32)

In [75]:
rt_pts

array([[[121.,  49.],
        [159.,  56.],
        [154.,  93.],
        [116.,  86.]],

       [[526., 133.],
        [554., 137.],
        [547., 167.],
        [520., 162.]],

       [[ 90., 276.],
        [128., 280.],
        [123., 314.],
        [ 85., 311.]],

       [[491., 322.],
        [518., 326.],
        [513., 354.],
        [486., 352.]],

       [[398., 189.],
        [423., 190.],
        [420., 215.],
        [395., 214.]],

       [[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[378., 349.],
        [403., 350.],
        [400., 375.],
        [375., 374.]],

       [[232., 263.],
        [257., 265.],
        [253., 289.],
        [229., 288.]]], dtype=float32)

In [76]:
rt_bord_pts

array([[121.,  49.],
       [159.,  56.],
       [154.,  93.],
       [116.,  86.],
       [526., 133.],
       [554., 137.],
       [547., 167.],
       [520., 162.],
       [ 90., 276.],
       [128., 280.],
       [123., 314.],
       [ 85., 311.],
       [491., 322.],
       [518., 326.],
       [513., 354.],
       [486., 352.]], dtype=float32)

In [77]:
rt_pro_pts

array([[398., 189.],
       [423., 190.],
       [420., 215.],
       [395., 214.],
       [378., 349.],
       [403., 350.],
       [400., 375.],
       [375., 374.],
       [232., 263.],
       [257., 265.],
       [253., 289.],
       [229., 288.]], dtype=float32)

In [79]:
def_pro_pts2

array([[185.,   7.],
       [214.,   7.],
       [214.,  36.],
       [185.,  36.],
       [185., 187.],
       [214., 187.],
       [214., 216.],
       [185., 216.],
       [  5.,  97.],
       [ 34.,  97.],
       [ 34., 126.],
       [  5., 126.]], dtype=float32)

In [78]:
def_pro_pts

array([[185.,   7.],
       [214.,   7.],
       [214.,  36.],
       [185.,  36.],
       [365.,  97.],
       [394.,  97.],
       [394., 126.],
       [365., 126.],
       [185., 187.],
       [214., 187.],
       [214., 216.],
       [185., 216.],
       [  5.,  97.],
       [ 34.,  97.],
       [ 34., 126.],
       [  5., 126.]], dtype=float32)

In [27]:
mat1


array([[ 9.60405407e-01,  1.22447867e-01, -2.31866518e+02],
       [-8.96626699e-02,  9.65837865e-01, -1.27093332e+02],
       [-1.65614493e-04, -2.59056563e-05,  1.00000000e+00]])

In [28]:
mat2

array([[ 1.39807564e+00, -1.58195591e-01,  1.24719333e+02],
       [ 2.95225205e-01,  1.30547548e+00,  3.89414812e+01],
       [ 4.99089657e-04,  1.56447773e-04,  1.00000000e+00]])

In [41]:
mat2, mask2 = cv2.findHomography(def_bord_pts, def_bord_pts, cv2.RANSAC,1)

In [42]:
mat2

array([[ 1.00000000e+00, -4.27609602e-18,  2.64898514e-14],
       [ 8.40553403e-18,  1.00000000e+00, -1.72414286e-14],
       [-1.61729948e-19, -2.01097451e-19,  1.00000000e+00]])

In [43]:
mat1, mask1 = cv2.findHomography(def_pro_pts, def_pro_pts, cv2.RANSAC,1)#初期設定（変更なし）

In [44]:
mat1

array([[ 1.00000000e+00,  2.35578812e-17, -2.00777132e-14],
       [ 3.56064849e-17,  1.00000000e+00,  1.56874126e-14],
       [ 5.37569556e-19, -1.43365630e-20,  1.00000000e+00]])